In [116]:
import pandas as pd
import re
import datetime
import seaborn as sns
from sklearn import preprocessing


In [117]:
df = pd.read_csv("data.csv")
#df

In [118]:
df.describe()

,Unnamed: 0,production_date,last_service,\nmanufacturer_color
count,62766.000000,6556.000000,0.0,0.0
mean,35563.827709,2014.969951,NaN,NaN
std,20562.958458,5.392130,NaN,NaN
min,0.000000,1997.000000,NaN,NaN
25%,17718.250000,2011.000000,NaN,NaN
50%,35595.500000,2016.000000,NaN,NaN
75%,53391.750000,2020.000000,NaN,NaN
max,71103.000000,2022.000000,NaN,NaN


# column name: location  # köy ve şehir isimlerinde işaret ile başlayan isimler için if döngüsü oluşturulacak.

In [119]:
df['location']=df['location'].apply(lambda x: re.sub(r"\\", "", x))# bir satirda '\\\\\\' ifadesi var onu kaldirdik
df['location'] = df['location'].apply(lambda x: str(x).upper()) # Tum harfleri buyuk harf yaptik
df['location'].value_counts()


location
ROTTERDAM, NL              1553
APELDOORN, NL              1142
AMSTERDAM, NL              1072
TILBURG, NL                 946
EINDHOVEN, NL               719
                           ... 
EINDHIVEN, NL                 1
OOSTERHESSELEN, NL            1
FERWERD, NL                   1
WELL, NL                      1
UTRECHTSE HEUVELRUG, NL       1
Name: count, Length: 1523, dtype: int64

# Column Name: Make

In [120]:
df["make"]


0         \n,Audi ,\n
1         \n,Audi ,\n
2         \n,Audi ,\n
3         \n,Audi ,\n
4         \n,Audi ,\n
             ...     
62761    \n,Volvo ,\n
62762    \n,Volvo ,\n
62763    \n,Volvo ,\n
62764    \n,Volvo ,\n
62765    \n,Volvo ,\n
Name: make, Length: 62766, dtype: object

In [121]:
df["make"]=df["make"].str.strip(',\n\r')
df['make'].head()

0    Audi 
1    Audi 
2    Audi 
3    Audi 
4    Audi 
Name: make, dtype: object

# Column Name: model

In [122]:
df["model"]


0         \n,A3,\n
1         \n,A3,\n
2         \n,A1,\n
3         \n,A1,\n
4         \n,A3,\n
           ...    
62761    \n,S60,\n
62762    \n,S60,\n
62763    \n,S60,\n
62764    \n,S40,\n
62765    \n,S60,\n
Name: model, Length: 62766, dtype: object

In [123]:
df["model"]=df["model"].str.strip(',\n\r')
df["model"].head()

0    A3
1    A3
2    A1
3    A1
4    A3
Name: model, dtype: object

# column name: short_description  # bu sütundaki değerler database kaydında hatırlatma için oluştuurlmuş değerler olabilir. Bundan dolayı gözardı edilebilir (şahsi fikrim :)

In [124]:
df['short_description']= df['short_description'].str.strip("\n,")
df['short_description'].value_counts()

short_description
1.4 TSI Highline                                      58
i                                                     51
i High Executive                                      45
i Executive                                           41
High Executive                                        36
                                                      ..
109 CDI | XL | Navigatie | Dubbele schuifdeur | De     1
110 CDI kort / rijklaar € 9450 ex btw / lease vana     1
210 2.2 CDI 366 HD L2 H2                               1
113 CDI 320 Lang Standaard / Airco / 3 Zits / Trek     1
2.4 Airco, Cruise Control, Stuurbekrachtiging          1
Name: count, Length: 54363, dtype: int64

# Column name: price

In [125]:
df['price'] = df['price'].apply(lambda x: '€ ' + re.sub(r'[^\d.,]+', '', x).replace(',', '.').replace('.', '', 1).rstrip('.'))
df['price'] = df['price'].str.replace('€', '')
pd.to_numeric(df['price'], errors='coerce').head()
# birim isaretleri kalkarsa rahatlikla mean median kullanabiliriz.

0     4.450
1    12.950
2    12.450
3    14.945
4    24.900
Name: price, dtype: float64

# Column Name : mileage    #dtype int olması gerekiyor

In [126]:
mileage_1 = []
for i in df['mileage']:
    result = re.search(r'\d{1,3}(?:,\d{3})*', i)
    if result:
        mileage_1.append(result.group(0))
    else:
        mileage_1.append(None)

mileage_son = pd.DataFrame(mileage_1,columns=["mileage"])

df["mileage"] = mileage_son["mileage"]

In [127]:
df.rename(columns = {'mileage':'mileage(km)'}, inplace = True)  # in place of every line, we write 'km' to column name as unit 
df["mileage(km)"].head()

0    191,066
1     94,758
2    145,102
3    119,666
4     55,815
Name: mileage(km), dtype: object

# colum name : gearbox  #Gearbox sütunu ile karşılaştırıldı 

In [128]:
df['gearbox'] = df['gearbox'].str.replace(r'[\d,]+', '', regex=True).str.strip()
df['gearbox'].value_counts()

# buyuk harfle yazilan daha iyi bir sutun


gearbox
Automatic           36633
Manual              25430
Semi-automatic        611
['\n' None '\n']       85
/                       7
Name: count, dtype: int64

# Gearbox sütunu ile karşılaştırıldı    # Büyük harf ile başlayan sütun kalacak diğer sütün drop yapılacak

In [129]:

df['Gearbox'] = df['Gearbox'].str.strip("\n ,")
df['gearbox'] = df['gearbox'].fillna("None")
df.loc[(df['gearbox'] != 'None'), ['gearbox', 'Gearbox']]
df['gearbox'].value_counts()

gearbox
Automatic           36633
Manual              25430
Semi-automatic        611
['\n' None '\n']       85
/                       7
Name: count, dtype: int64

# colum name : registration

In [130]:
df['registration'].value_counts()

registration
01/2021    2883
01/2022    1284
12/2016     719
03/2021     719
12/2020     636
           ... 
05/1997       1
02/1981       1
05/1995       1
07/2028       1
10/1996       1
Name: count, Length: 302, dtype: int64

# Column name : fuel_type  #Fuel_type sütunu ile karşılaştırıldı. #Fuel_type sütunu kalacak diğer sütun silinebilir. İki sütun karşılaştırılmasında fuel_type da yer alan None değerlerin karşılığı Fuel_type sütununda electric ile eşleştiği için Fuel_type sütunundaki None değerler Electric ile doldurulacak.

In [131]:
df['fuel_type'] = df['fuel_type'].str.strip("\n\r,")
df['Fuel_type'] = df['Fuel_type'].str.strip("\n\r,")
df['fuel_type'] = df['fuel_type'].fillna("None")
df.loc[(df['fuel_type'] != 'None'), ['fuel_type', 'Fuel_type']]
df['fuel_type'].value_counts() # Icinde gearbox ile lgili bilgiler de var. Kayit ederken kaydirma yapilmis olabilir
# fuel_type i secmek digerini silmek mantkli olabilir
df['Fuel_type'].value_counts()

Fuel_type
Gasoline                                        22959
Super 95                                        17686
Diesel                                          10783
None                                             8782
Diesel (Particle filter)                         1209
Super E10 95                                      408
LPG                                               309
Liquid petroleum gas (LPG)                        237
Gasoline (Particle filter)                        140
Electric                                           74
Domestic gas L                                     56
CNG                                                41
Super Plus 98                                      27
Super 95 (Particle filter)                         24
Hydogen                                            12
Ethanol                                             9
Regular/Benzine 91                                  4
Others                                              3
Electric (Particle

# Column name : power     #Power sütunu ile karşılaştırıldı. hp birim olarak kullanılmasının uygun olduğuna karar verildi. Birim ifadesi sütuna eklenecek.

In [132]:
df['power'] = df['power'].str.strip("\n\r,")
df['Power'] = df['Power'].str.strip("\n\r,")
df['power'] = df['power'].fillna("None")
df.loc[(df['power'] != 'None'), ['power', 'Power']]
# Integere cevirmek icin birimleri kolona alabiliriz. hp zaten kW ye bagli bi deger oldugu icin o kismi rahatlikla silebiliriz
df['Power'].value_counts() # Buyuk harfle olani almak daha mantikl olabilir
df['Power'] = df['Power'].str.extract(r'(\d+)')
df['Power'] # sutun ismini tum analizleri yaptiktan sonra degstiririz.

0        110
1         92
2         92
3         92
4        110
        ... 
62761    125
62762    103
62763    132
62764    103
62765    104
Name: Power, Length: 62766, dtype: object

# Column Name : seller

In [133]:
df['seller'] = df['seller'].str.lstrip("[', [' \s\S\t n ")
df['seller'] = df['seller'].str.rstrip("[', [] ' \s\S\t n")  


#seller has three not related unique values(electric, diesel , Gasoline) 
#bunlar diger sutunlara ait olabilir
df["seller"].replace("Diesel", None, inplace = True) 
df["seller"].replace("Electric", None, inplace = True) 
df["seller"].replace("Gasoline", None, inplace = True) 
df["seller"].value_counts()

seller
Dealer            57428
Private seller     5330
Name: count, dtype: int64

# Column name: body_type   # None değerlerin sayısı düşük  ne yapılacağına karar verilecek.

In [134]:
df['body_type'] = df['body_type'].str.replace(',', '').str.strip()
df['body_type'].value_counts() # 948 none deger

body_type
Off-Road/Pick-up    17348
Compact             13185
Station wagon       11318
Van                  6243
Sedan                5341
Convertible          3645
Transporter          2249
Coupe                1923
None                  948
Other                 562
Panel van               2
Car transport           1
Flatbed van             1
Name: count, dtype: int64

# Column name: type

In [135]:
df['type'] = df['type'].str.replace(',', '').str.strip()
df['type'].value_counts()# 948 none values

type
Used              55523
New                3413
Demonstration      1889
Pre-registered      993
None                948
Name: count, dtype: int64

# Column name: drivetrain  # drivetrain ile ilgili literatür calismasi yapilacak. 14594 none değeri tarama sonrası değerlendirilecek. 

In [136]:
df['drivetrain'] = df['drivetrain'].str.replace(',', '').str.strip()
df['drivetrain'].value_counts() # cekis demek , 14594 none var

drivetrain
Front    33522
None     14594
4WD       7363
Rear      7287
Name: count, dtype: int64

# Column Name: seats   #Sütundaki 0 değerlerini ortalama 5 koltuk olduğundan dolayı 5 ile doldurabiliriz.

In [137]:
df["seats"]

0        \n5
1        \n5
2        \n5
3        \n4
4        \n5
        ... 
62761    \n5
62762    \n5
62763    \n5
62764    \n5
62765    \n5
Name: seats, Length: 62766, dtype: object

In [138]:
df["seats"]=df["seats"].str.strip(',\n')

In [139]:
df["seats"].replace("None", "0", inplace = True) #None values may stay as None. There are no car without seat

In [140]:
df["seats"]=df["seats"].astype(int)

df["seats"].value_counts()

seats
5     46671
4      6057
0      4925
2      1995
7      1846
3       899
6       257
8        57
9        56
1         2
14        1
Name: count, dtype: int64

# Column Name: doors  # kapı sayısındaki 0 değerleri ortalama değer ile doldurulabilir.

In [141]:
df["doors"].unique()

array(['\n3', '\n5', '\n4', 'None\n', '\n2', '\n6', '\n1'], dtype=object)

In [142]:

df["doors"]=df["doors"].str.strip(',\n')
df["doors"].replace("None", "0", inplace = True) #None values may stay as None. There are no car without door
df["doors"]=df["doors"].astype(int)
df["doors"].value_counts()

doors
5    45496
4     8413
2     5286
3     2329
0     1161
6       79
1        2
Name: count, dtype: int64

# Column Name: country_version   #54824 None değeri var. Değer çok yüksek?

In [143]:
df['country_version'] = df['country_version'].str.split(',').str[-1].str.strip()
df['country_version'].value_counts()
# 54824 none values var

country_version
None                    54824
Netherlands              6835
Germany                   772
Belgium                   106
Sweden                     50
Denmark                    36
France                     21
Czechia                    19
Spain                      16
Switzerland                14
Italy                      13
United States              13
Poland                     12
Austria                     9
Luxembourg                  6
Slovenia                    5
United Kingdom              2
Hungary                     2
Japan                       2
Canada                      2
Romania                     1
Slovakia                    1
Estonia                     1
Namibia                     1
Lithuania                   1
Latvia                      1
United Arab Emirates        1
Name: count, dtype: int64

# Column name : offer_number #fatura numarası veya sipariş numarası olup olmadığı literatür araştırması yapılarak karar verilecek.

In [144]:
df["offer_number"]= df["offer_number"].str.strip(',\n')
df["offer_number"].value_counts()

offer_number
None        9928
NIEUW         53
XX            28
.             17
1776          13
            ... 
#38416         1
#38787         1
#38418         1
#37783         1
26-HN-LS       1
Name: count, Length: 50143, dtype: int64

# Column name : warranty # literatür taraması yapılacak. sonrasında değerlendirilecek

In [145]:
df['warranty'] = df['warranty'].str.replace(',', '').str.strip()
df['warranty'].value_counts()

warranty
None         41605
6 months     16607
3 months      1643
12 months     1362
24 months     1026
1 months       248
84 months      204
18 months       35
36 months       10
2 months         8
48 months        6
Yes              3
60 months        3
26 months        2
9 months         2
10 months        1
4 months         1
Name: count, dtype: int64

# Column name : Mileage    #Data içinde 2 adet Mileage sütunu mevcut ve bu sütunda bilgi olmadığından drop yapılacak

In [146]:
df['Mileage'].unique()

array(["[['\\n'], ['']]", "[['\\n'], ['None']]"], dtype=object)

# Column name: first_registration

In [147]:
df['first_registration'].head()


0    08/2003
1    10/2013
2    06/2015
3    10/2015
4    10/2017
Name: first_registration, dtype: object

# Column Name : production_date  ve registration değerleri birbiri ile uyumlu

In [148]:
# df['production_date'] = pd.to_datetime(df['production_date'], format='%Y')

#Producton_date ve registration arasinda bag var. Yillar ayni. Burdan doldurabiliriz
df['production_date'] = df['production_date'].fillna("None")
df.loc[(df['production_date'] != 'None'), ['production_date', 'registration']]


,production_date,registration
2,2015.0,06/2015
7,2011.0,07/2011
8,2012.0,08/2012
14,2012.0,05/2012
26,2013.0,07/2013
...,...,...
62696,2007.0,03/2007
62708,2010.0,09/2010
62714,2007.0,07/2007
62719,2005.0,02/2005


# Column Name : general_inspection #15532 new değeri mevcut. New Değerleri 2023 olarak doldurulacak.

In [149]:
df["general_inspection"].value_counts()
#df["general_inspection"].unique()

general_inspection
New        15532
06/2022     2169
03/2022     2025
07/2022     1936
05/2022     1930
           ...  
11/2014        1
09/2017        1
08/2018        1
06/2019        1
12/2019        1
Name: count, Length: 93, dtype: int64

# Column Name : last_service               #column içerisinde sadece nan değeri var. Drop yapılacak. Kalıcı silme olmayacak.

In [150]:
df['last_service'].unique()

array([nan])

# Column Name: full_service_history  #dtype int cevrilecek. 0 ve 1 olarak değiştirilecek.

In [151]:
df['full_service_history'] = df['full_service_history'].str.replace(',', '').str.strip()
df['full_service_history'].unique()


array(['Yes', 'None'], dtype=object)

# full_service_encoded sütununda değerler 1-0 olarak yazıldı. 

In [152]:
from sklearn.preprocessing import LabelEncoder          
lbe=LabelEncoder()
lbe.fit_transform(df["full_service_history"])

df['full_service_encoded']=lbe.fit_transform(df["full_service_history"])
df['full_service_encoded']

0        1
1        1
2        1
3        1
4        1
        ..
62761    1
62762    1
62763    1
62764    0
62765    0
Name: full_service_encoded, Length: 62766, dtype: int32

# Column Name : non_smoker_vehicle #dtype int cevrilecek. 0 ve 1 olarak değiştirilecek.

In [153]:
df['non_smoker_vehicle'] = df['non_smoker_vehicle'].str.replace(',', '').str.strip()
df['non_smoker_vehicle'].value_counts()

non_smoker_vehicle
None    57769
Yes      4997
Name: count, dtype: int64

# full_service_encoded sütununda değerler 1-0 olarak yazıldı. 

In [154]:
lbe.fit_transform(df["non_smoker_vehicle"])

array([0, 0, 1, ..., 0, 0, 0])

In [155]:
df['non_smoker_encoded']=lbe.fit_transform(df["non_smoker_vehicle"])
df['non_smoker_encoded']

0        0
1        0
2        1
3        0
4        0
        ..
62761    0
62762    0
62763    0
62764    0
62765    0
Name: non_smoker_encoded, Length: 62766, dtype: int32

# Column Name : previous_owner

In [156]:
df['previous_owner'] = df['previous_owner'].str.lstrip("[' \s\S\t n ',]")
df['previous_owner'] = df['previous_owner'].str.rstrip("['\s,]")
df['previous_owner'].unique()

array(['None', '3', '2', '1', '5', '7', '9', '4', '6', '8', '12', '10'],
      dtype=object)

# Column Name : last_timing_belt_change  # 61925 None değeri hic degismedigi anlamina geldigi icin production date ile doldurulacak.

In [157]:
df['last_timing_belt_change'] = df['last_timing_belt_change'].str.lstrip(" [',] '\a \s\S\t\r n ' ")
df['last_timing_belt_change'] = df['last_timing_belt_change'].str.rstrip(" [',] '\a \s\S\t\r n ' ")

df['last_timing_belt_change'].unique()
df['last_timing_belt_change'].value_counts()

last_timing_belt_change
None       61925
06/2021       41
05/2021       31
10/2021       31
09/2021       28
           ...  
02/2015        1
04/2016        1
06/2014        1
04/2015        1
11/2015        1
Name: count, Length: 82, dtype: int64

# Column Name : Power   #dtype int e cevrilecek. 5333 None değeri var

In [158]:
df["Power"].value_counts()

Power
110    3677
88     2518
100    2190
90     2175
85     2083
       ... 
378       1
164       1
237       1
37        1
10        1
Name: count, Length: 355, dtype: int64

In [159]:
df["power"].value_counts()

power
[None, '']         5333
110 kW (150 hp)    3677
88 kW (120 hp)     2518
100 kW (136 hp)    2190
90 kW (122 hp)     2174
                   ... 
378 kW (514 hp)       1
282 kW (383 hp)       1
470 kW (639 hp)       1
451 kW (613 hp)       1
10 kW (14 hp)         1
Name: count, Length: 361, dtype: int64

# Column Name: gearbox   aynı sütundan var

# Column Name : engine_size   # dtype int e cevrilecek. 5143 None değeri var

In [160]:
df["engine_size"]= df["engine_size"].str.strip(',\n,')
df["engine_size"]= df["engine_size"].replace("None", "0")
df["engine_size"]= df["engine_size"].str.replace("cc", "")

df["engine_size"].unique()

array(['1,984 ', '1,395 ', '1,495 ', '999 ', '1,390 ', '1,798 ', '1,197 ',
       '3,197 ', '2,773 ', '2,000 ', '1,399 ', '1,398 ', '3,189 ',
       '2,995 ', '3,199 ', '1,595 ', '2,480 ', '1,498 ', '1,781 ',
       '1,998 ', '1,500 ', '3,993 ', '3,184 ', '1,598 ', '1,800 ', '0',
       '2,393 ', '2,998 ', '4,163 ', '3,123 ', '3,195 ', '3,000 ',
       '2,976 ', '5,204 ', '1,997 ', '2,496 ', '1,968 ', '5,198 ',
       '2,894 ', '4,198 ', '1,400 ', '2,967 ', '3,597 ', '4,172 ',
       '2,671 ', '3,996 ', '1,000 ', '3,956 ', '2,700 ', '1,801 ',
       '1,995 ', '4,000 ', '2,771 ', '4,200 ', '5,998 ', '3,697 ',
       '3,200 ', '0 ', '1,799 ', '2,698 ', '4,134 ', '1,422 ', '1,191 ',
       '1,896 ', '3,001 ', '5,934 ', '1 ', '2,001 ', '1,964 ', '1,796 ',
       '1,499 ', '2,979 ', '1,599 ', '1,597 ', '4,395 ', '2,996 ', '4 ',
       '1,596 ', '2,494 ', '1,600 ', '3,246 ', '2,497 ', '4,398 ',
       '4,799 ', '4,999 ', '2,993 ', '2,171 ', '1,991 ', '2,200 ',
       '2,793 ', '4,941 ', '2,4

In [161]:
df["engine_size"] = df["engine_size"].astype(int)

ValueError: invalid literal for int() with base 10: '1,984 '

# Column Name : gears  # 18184 none degeri var literatür arastirmasi yapilacak  

In [ ]:
df["gears"]=df["gears"].str.strip(',\n,')
df["gears"].unique()
df["gears"].value_counts()

gears
6       18882
None    18184
5       10834
7        6867
8        4905
1        1547
9         881
4         621
2          37
3           8
Name: count, dtype: int64

bi iki koda gerek var mi pek emin degilim,
Data missing de konusabiliriz!!!
df["gears"].replace("None", "0", inplace = True)
df["gears"]=df["gears"].astype(int)

# Column Name: cylinders  5532 None değeri var

In [ ]:
df["cylinders"]=df["cylinders"].str.strip(',\n,')
df["cylinders"].unique()
df["cylinders"].value_counts()

cylinders
4       41993
3        7664
None     5532
6        4260
5        1390
8         937
1         498
2         400
10         65
12         26
7           1
Name: count, dtype: int64

# Column Name : empty_weight

In [ ]:
df["empty_weight"]=df["empty_weight"].str.strip('[\n,')
df["empty_weight"]=df["empty_weight"].str.replace(' kg', '')
df["empty_weight"].value_counts()

empty_weight
1,295    1529
None     1407
1,405    1127
1,635     795
1,065     658
         ... 
2,387       1
2,481       1
2,163       1
2,283       1
2,036       1
Name: count, Length: 1539, dtype: int64

# Column Name: Fuel_type

In [ ]:
df["Fuel_type"]=df["Fuel_type"].str.strip('[\n,')
df["Fuel_type"].head(15)

0     Super 95
1     Gasoline
2     Gasoline
3     Gasoline
4     Super 95
5     Super 95
6     Gasoline
7     Gasoline
8     Gasoline
9     Super 95
10    Gasoline
11    Super 95
12    Gasoline
13    Super 95
14    Gasoline
Name: Fuel_type, dtype: object

# Column name : fuel_consumption # city ve country sütunlari olusturulacak.

In [ ]:
df['fuel_consumption'] = df['fuel_consumption'].str.lstrip(" [', \s\S\t\r\n,n ' ")
df['fuel_consumption'] = df['fuel_consumption'].str.rstrip("[', '] \s\S\t\r\n,\\n  ")
df['fuel_consumption'] = df['fuel_consumption'].str.replace("'","")
df['fuel_consumption']

0                                                     None
1                                     4.9 l/100 km (comb.)
2        0 l/100 km (comb.), 0 l/100 km (city), 0 l/100...
3        4.8 l/100 km (comb.), 5.9 l/100 km (city), 4.2...
4        5 l/100 km (comb.), 6.2 l/100 km (city), 4.3 l...
                               ...                        
62761                                                 None
62762                                                 None
62763    9.1 l/100 km (comb.), 12.6 l/100 km (city), 7 ...
62764                                                 None
62765    8.4 l/100 km (comb.), 11.6 l/100 km (city), 6....
Name: fuel_consumption, Length: 62766, dtype: object

# Column Name: co2_emissions

In [ ]:
df['co2_emissions'] = df['co2_emissions'].str.strip(",\n")
df["co2_emissions"] = df["co2_emissions"].str.rstrip(" [',] \s\S\t\r\n n' ")
df["co2_emissions"] = df["co2_emissions"].str.lstrip(" [', \s\S\t\r\n ' ")
df['co2_emissions'].value_counts()


co2_emissions
None                9867
                    6887
0 g/km (comb.)      4973
139 g/km (comb.)    1061
149 g/km (comb.)     911
                    ... 
392 g/km (comb.)       1
378 g/km (comb.)       1
24 g/km (comb.)        1
55 g/km (comb.)        1
70 g/km (comb.)        1
Name: count, Length: 346, dtype: int64

# Column Name : emission_class  #literatür incelenecek ve euro 6 için euro 6 c veya euro6d olup olmadığı araştırılacak eğer yoksa euro 6c ve 6d değerleri Euro 6 şeklinde düzenlenecek

In [ ]:
df['emission_class'] = df['emission_class'].str.strip(",\n")
df['emission_class'].value_counts()

emission_class
Euro 6                           19704
None                             19076
Euro 5                           10675
Euro 4                            7710
Euro 6d-TEMP                      2675
Euro 3                            1702
['\n', [' ', ' ', ' '], '\n']      597
Euro 1                             461
Euro 2                             129
Euro 6c                             20
Euro 6d                             17
Name: count, dtype: int64

# Column Name: emissions_sticker           #drop yapılacak

In [ ]:
df["emissions_sticker"].unique()

array(['\n,None,\n', '\n,4 (Green),\n', '\n,3 (Yellow),\n',
       '\n,1 (No sticker),\n', "['\\n', [' ', ' ', ' '], '\\n']",
       '\n,2 (Red),\n'], dtype=object)

In [ ]:
df["emissions_sticker"] = df["emissions_sticker"].str.rstrip(" [',] \s\S\t\r\n ' ")
df["emissions_sticker"] = df["emissions_sticker"].str.lstrip(" [',] \s\S\t\r\n ' ")
df["emissions_sticker"] = df["emissions_sticker"].str.strip(" ")
df["emissions_sticker"] = df["emissions_sticker"].str.replace("\\n","")
df["emissions_sticker"].value_counts()

emissions_sticker
None                      62632
4 (Green)                   106
1 (No sticker)               12
n', [' ', ' ', ' '], '       11
3 (Yellow)                    4
2 (Red)                       1
Name: count, dtype: int64

# Column Name : Comfort & Convenience   #Air Conditioning -Cruise control sütunları oluşturulacak ve 0-1 şeklinde düzenlenecek.

In [ ]:
df.rename({"\nComfort & Convenience\n":"comfort_convenience"}, inplace = True, axis = 1) # sütun isminde \n değerleri olduğu için kalıcı oalrak sütun ismi değiştirildi.
df["comfort_convenience"] = df["comfort_convenience"].str.rstrip(" [', \s\S\t\r n ' ")
df["comfort_convenience"] = df["comfort_convenience"].str.lstrip(" [', '\a \s\S\t\r n ' ")
df["comfort_convenience"] = df["comfort_convenience"].str.replace("'","")
df["comfort_convenience"].unique()

array(['Air conditioning, Armrest, Automatic climate control, Electrical side mirrors, Navigation system, Power windows, Sunroof]]',
       'Air conditioning, Automatic climate control, Electrical side mirrors, Light sensor, Power windows, Rain sensor]]',
       'Air conditioning, Armrest, Automatic climate control, Cruise control, Navigation system, Parking assist system sensors rear, Power windows, Rain sensor, Seat heating, Split rear seats]]',
       ...,
       'Air conditioning, Automatic climate control, Cruise control, Electrical side mirrors, Electrically adjustable seats, Leather steering wheel, Parking assist system sensors rear, Rain sensor, Seat heating]]',
       'Air conditioning, Automatic climate control, Cruise control, Electrical side mirrors, Leather steering wheel, Parking assist system sensors rear, Power windows, Rain sensor, Sunroof]]',
       'Air conditioning, Automatic climate control, Cruise control, Electrical side mirrors, Leather steering wheel, Navigatio

In [ ]:


df['air_conditioning'] = df['comfort_convenience'].str.split(',').str.get(0)

df['air_conditioning'].head()

#df['air_conditioning1'] = df['air_conditioning1'].fillna("None")
#df.loc[(df['air_conditioning1'] != 'None'), ['air_conditioning1', 'air_conditioning']]

0    Air conditioning
1    Air conditioning
2    Air conditioning
3    Air conditioning
4    Air conditioning
Name: air_conditioning, dtype: object

In [ ]:
lbe.fit_transform(df["air_conditioning"])

array([1, 1, 1, ..., 1, 1, 1])

In [ ]:
df['air_conditioning']=lbe.fit_transform(df["air_conditioning"])
df['air_conditioning'].head()

0    1
1    1
2    1
3    1
4    1
Name: air_conditioning, dtype: int32

# Column Name : Entertaiment & Media  #Bluetooth sütunu oluşturulacak 0 ve 1 şeklinde düzenlenecek

In [ ]:
df['\nEntertainment & Media\n'].unique()

array(["[['CD player', 'Radio']]",
       "[['Bluetooth', 'CD player', 'On-board computer', 'USB']]",
       "[['CD player', 'MP3', 'On-board computer', 'Radio']]", ...,
       "[['Apple CarPlay', 'CD player', 'Digital cockpit', 'On-board computer', 'Radio']]",
       "[['Apple CarPlay', 'CD player', 'On-board computer', 'Radio', 'Sound system']]",
       "[['Android Auto', 'Apple CarPlay', 'Bluetooth', 'CD player', 'Digital cockpit', 'Digital radio', 'Hands-free equipment', 'Induction charging for smartphones', 'Integrated music streaming', 'On-board computer', 'Radio', 'Sound system', 'USB']]"],
      dtype=object)

In [ ]:
df.rename({"\nEntertainment & Media\n":"entertainment_media"},inplace = True, axis = 1)# sütun isminde \n değerleri olduğu için kalıcı oalrak sütun ismi değiştirildi.
df["entertainment_media"] = df["entertainment_media"].str.rstrip(" [',] \s\S\t\r n ' ")
df["entertainment_media"] = df["entertainment_media"].str.lstrip(" [', '\a \s\S\t\r n ' ")
df["entertainment_media"] =  df["entertainment_media"].str.replace("'","")
df["entertainment_media"].unique()

array(['CD player, Radio', 'Bluetooth, CD player, On-board computer, USB',
       'CD player, MP3, On-board computer, Radio', ...,
       'Apple CarPlay, CD player, Digital cockpit, On-board computer, Radio',
       'Apple CarPlay, CD player, On-board computer, Radio, Sound system',
       'Android Auto, Apple CarPlay, Bluetooth, CD player, Digital cockpit, Digital radio, Hands-free equipment, Induction charging for smartphones, Integrated music streaming, On-board computer, Radio, Sound system, USB'],
      dtype=object)

In [ ]:
search_value = 'Bluetooth'

# sütunun içinde arama yaparak ilgili satırları seçmek
search_result = df[df['entertainment_media'].str.contains(search_value)]

# arama sonucunu yazdırmak
if not search_result.empty:
    print(f"Aradığınız sayı ({search_value}) veri kümesinde yer alan satırlarda mevcut:")
    print(search_result)
else:
    print(f"Aradığınız sayı ({search_value}) veri kümesinde yer alan satırlarda bulunamadı.")

Aradığınız sayı (Bluetooth) veri kümesinde yer alan satırlarda mevcut:
       Unnamed: 0                    location    make model   
2               3           BAARLE-NASSAU, NL   Audi     A1  \
7              10                  HEESCH, NL   Audi     A3   
8              11  VALKENBURG AAN DE GEUL, NL   Audi     A1   
17             22               OLDENZAAL, NL   Audi     A5   
20             26              RIDDERKERK, NL   Audi     A3   
...           ...                         ...     ...   ...   
62691       71023                ENSCHEDE, NL  Volvo    V60   
62708       71042              LOENEN GLD, NL  Volvo    V70   
62714       71048                 HENGELO, NL  Volvo    V70   
62728       71064                   ASSEN, NL  Volvo    V70   
62755       71092                  ZETTEN, NL  Volvo    S80   

                                       short_description    price mileage(km)   
2        1.4 TFSI Sportback Pro-Line Aut. S-Tronic Navi.   12.450     145,102  \
7         

# Column Name : Safety & Security

In [164]:
df.rename({"\nSafety & Security\n":"safety_security"},inplace = True, axis = 1)# sütun isminde \n değerleri olduğu için kalıcı oalrak sütun ismi değiştirildi.
df["safety_security"] = df["safety_security"].str.rstrip(" [', \s\S\t\r n ' ")
df["safety_security"] = df["safety_security"].str.lstrip(" [', '\a \s\S\t\r n ' ")
df["safety_security"] =  df["safety_security"].str.replace("'","")
df["safety_security"].unique()

array(['ABS, Alarm system, Central door lock, Central door lock with remote control, Driver-side airbag, Electronic stability control, Fog lights, Head airbag, Immobilizer, Passenger-side airbag, Power steering, Side airbag, Traction contro]]',
       'Alarm system, Central door lock, Central door lock with remote control, Fog lights, Immobilizer]]',
       'ABS, Central door lock with remote control, Driver-side airbag, Electronic stability control, Fog lights, Head airbag, Immobilizer, Passenger-side airbag, Power steering, Side airbag, Tire pressure monitoring system, Traction contro]]',
       ...,
       'ABS, Adaptive headlights, Alarm system, Bi-Xenon headlights, Central door lock with remote control, Daytime running lights, Driver-side airbag, Electronic stability control, Fog lights, Head airbag, Isofix, Passenger-side airbag, Power steering, Rear airbag, Side airbag, Traction contro]]',
       'ABS, Central door lock with remote control, Driver-side airbag, Electronic stabili

In [177]:
search_value = 'ABS'

# sütunun içinde arama yaparak ilgili satırları seçmek
search_result = df[df['safety_security'].str.contains(search_value)]

# arama sonucunu yazdırmak
if not search_result.empty:
    print(f"Aradığınız sayı ({search_value}) veri kümesinde yer alan satırlarda mevcut:")
    print(search_result)
else:
    print(f"Aradığınız sayı ({search_value}) veri kümesinde yer alan satırlarda bulunamadı.")

Aradığınız sayı (ABS) veri kümesinde yer alan satırlarda mevcut:
       Unnamed: 0                 location    make model   
0               0         LOON OP ZAND, NL   Audi     A3  \
2               3        BAARLE-NASSAU, NL   Audi     A1   
3               4              ZUTPHEN, NL   Audi     A1   
4               7          HAAKSBERGEN, NL   Audi     A3   
5               8                WEZEP, NL   Audi     A3   
...           ...                      ...     ...   ...   
62761       71098           ETTEN-LEUR, NL  Volvo    S60   
62762       71099  HENDRIK-IDO-AMBACHT, NL  Volvo    S60   
62763       71101             LELYSTAD, NL  Volvo    S60   
62764       71102            AMSTERDAM, NL  Volvo    S40   
62765       71103          SOESTERBERG, NL  Volvo    S60   

                                       short_description    price mileage(km)   
0                              2.0 FSI Ambition Pro Line    4.450     191,066  \
2        1.4 TFSI Sportback Pro-Line Aut. S-Tronic N

In [181]:
df['abs_encoded'] = df['safety_security'].str.split(',').str.get(0)

df['abs_encoded'].unique()

array(['ABS', 'Alarm system', 'Adaptive Cruise Control',
       'Bi-Xenon headlights', 'Central door lock',
       'Central door lock with remote control', 'Daytime running lights',
       'Isofix', 'Driver-side airbag', 'Adaptive headlights',
       'Blind spot monitor', 'Xenon headlights]]', 'Power steering]]',
       'Isofix]]', 'Immobilizer',
       'Central door lock with remote contro]]',
       'Driver drowsiness detection', 'LED Daytime Running Lights',
       'High beam assist', 'Bi-Xenon headlights]]', 'ABS]]',
       'Power steering', 'Tire pressure monitoring system]]',
       'Alarm system]]', 'Electronic stability control',
       'Emergency system', 'Fog lights', 'Driver-side airbag]]',
       'Immobilizer]]', 'Fog lights]]', 'Adaptive Cruise Contro]]',
       'Central door lock]]', 'LED Headlights]]', 'Blind spot monitor]]',
       'Head airbag', 'Distance warning system',
       'Emergency brake assistant]]', 'LED Headlights',
       'Emergency system]]', 'Lane departu

In [182]:
lbe.fit_transform(df["abs_encoded"])

array([0, 5, 0, ..., 0, 0, 0])

In [183]:
df['abs']=lbe.fit_transform(df["abs_encoded"])
df['abs'].head()

0    0
1    5
2    0
3    0
4    0
Name: abs, dtype: int32

# Column Name : extras

In [ ]:
df["extras"] = df["extras"].str.rstrip(" [',] \s\S\t\r n ' ")
df["extras"] = df["extras"].str.lstrip(" [',] '\a \s\S\t\r n ' ")
df["extras"] =  df["extras"].str.replace("'","")
df["extras"]

0                                              Alloy wheel
1                                       Alloy wheels (16")
2                                       Alloy wheels (15")
3        Alloy wheels (17"), Shift paddles, Sport seats...
4         Alloy wheels (17"), Sport seats, Sport suspensio
                               ...                        
62761                                   Alloy wheels (15")
62762                                   Alloy wheels (15")
62763    Alloy wheels (17"), Automatically dimming inte...
62764    Alloy wheels (16"), Automatically dimming inte...
62765                                          Alloy wheel
Name: extras, Length: 62766, dtype: object

# Column Name: manifacturer_colour    içeriği tamamen nan değeriyle dolu

In [ ]:
df['\nmanufacturer_color'].unique()

array([nan])

# Column Name: colour

In [ ]:
df.rename(columns = {'\ncolour':'colour'}, inplace = True)
df["colour"].unique()

array(['\n4, Grey', '\n4, White', '\n4, Black', '\n4, Red', '\n4, Silver',
       '\n4, Brown', '\n4, Blue', '\n4, Violet', nan, '\n4, Beige',
       '\n4, Yellow', '\n4, Green', '\n4, Orange', '\n4, Bronze',
       '\n4, Gold'], dtype=object)

In [ ]:
df["colour"]=df["colour"].str.strip(',\n4,')
df["colour"].head()
#1048 none var

0      Grey
1     White
2      Grey
3     Black
4      Grey
Name: colour, dtype: object

# Column Name : paint   (arabanın boyanıp boyanmadığını ifade ediyor. Renk belirtilenler boyanmış belirtilmeyenler boyanmamış. Bundan dolayı rengin yanındaki paint ifadesini gözardı edebiliriz.)

In [ ]:
df.rename(columns = {'\npaint':'paint'}, inplace = True)
df["paint"].unique()
df['paint']


0         Metallic
1              NaN
2         Metallic
3              NaN
4         Metallic
           ...    
62761          NaN
62762          NaN
62763     Metallic
62764     Metallic
62765     Metallic
Name: paint, Length: 62766, dtype: object

In [ ]:
df["paint"]=df["paint"].str.replace('\n','')
df["paint"].head()

0     Metallic
1          NaN
2     Metallic
3          NaN
4     Metallic
Name: paint, dtype: object

In [ ]:
df

# Column Name : upholstery_colour

In [ ]:
df[' upholstery_colour '].unique()

array(['\nupholstery, Black', nan, '\nupholstery, Grey',
       '\nupholstery, Brown', '\nupholstery, Other',
       '\nupholstery, Beige', '\nupholstery, Red', '\nupholstery, Yellow',
       '\nupholstery, Blue', '\nupholstery, White', '\nupholstery, Green',
       '\nupholstery, Orange'], dtype=object)

In [ ]:
df.rename(columns={' upholstery_colour ':'upholstery_colour'},inplace = True)
df["upholstery_colour"]=df["upholstery_colour"].str.replace('\nupholstery,','')
df["upholstery_colour"].unique()

array([' Black', nan, ' Grey', ' Brown', ' Other', ' Beige', ' Red',
       ' Yellow', ' Blue', ' White', ' Green', ' Orange'], dtype=object)

# Column Name : upholstery

In [ ]:
df[" upholstery "].unique()

array(['\n8, Cloth', nan, '\n8, Part leather', '\n8, Full leather',
       '\n8, alcantara', '\n8, Other', '\n8, Velour'], dtype=object)

In [ ]:
df.rename(columns={' upholstery ':'upholstery'},inplace = True)
df["upholstery"]=df["upholstery"].str.replace('\n8,','')
df["upholstery"].unique()

array([' Cloth', nan, ' Part leather', ' Full leather', ' alcantara',
       ' Other', ' Velour'], dtype=object)

In [ ]:
#df['registration'] = df['fuel_type'].str.strip("\n\r,")
#df['Fuel_type'] = df['Fuel_type'].str.strip("\n\r,")
#df['registration'] = df['registration'].fillna("None")
df.loc[(df['production_date'] != 'None'), ['registration', 'production_date']].head(50)
#df['fuel_type'].value_counts() # Icinde gearbox ile lgili bilgiler de var. Kayit ederken kaydirma yapilmis olabilir
# fuel_type i secmek digerini silmek mantkli olabilir


,registration,production_date
0,08/2003,NaN
1,10/2013,NaN
2,06/2015,2015.0
3,10/2015,NaN
4,10/2017,NaN
5,03/2019,NaN
6,03/2014,NaN
7,07/2011,2011.0
8,08/2012,2012.0
9,01/2011,NaN


# Column name : Mileage    #Data içinde 2 adet Mileage sütunu mevcut ve bu sütunda bilgi olmadığından drop yapılacak

In [ ]:
df['Mileage'].unique()

array(["[['\\n'], ['']]", "[['\\n'], ['None']]"], dtype=object)

# Column name: first_registration

In [ ]:
df['first_registration'].head()


0    08/2003
1    10/2013
2    06/2015
3    10/2015
4    10/2017
Name: first_registration, dtype: object

# Column Name : production_date  ve registration değerleri birbiri ile uyumlu

In [ ]:
# df['production_date'] = pd.to_datetime(df['production_date'], format='%Y')

#Producton_date ve registration arasinda bag var. Yillar ayni. Burdan doldurabiliriz
df['production_date'] = df['production_date'].fillna("None")
df.loc[(df['production_date'] != 'None'), ['production_date', 'registration']]


,production_date,registration
2,2015.0,06/2015
7,2011.0,07/2011
8,2012.0,08/2012
14,2012.0,05/2012
26,2013.0,07/2013
...,...,...
62696,2007.0,03/2007
62708,2010.0,09/2010
62714,2007.0,07/2007
62719,2005.0,02/2005


# Column Name : general_inspection #bu veride ne yapılacak

In [ ]:
df["general_inspection"].value_counts()
#df["general_inspection"].unique()

general_inspection
New        15532
06/2022     2169
03/2022     2025
07/2022     1936
05/2022     1930
           ...  
11/2014        1
09/2017        1
08/2018        1
06/2019        1
12/2019        1
Name: count, Length: 93, dtype: int64

# Column Name : last_service               #column içerisinde sadece nan değeri var

In [ ]:
df['last_service'].unique()

array([nan])

# Column Name: full_service_history   # dtype int e çevrilecek ve 0-1 olarak düzenlenecek

In [29]:
df['full_service_history'] = df['full_service_history'].str.replace(',', '').str.strip()
df['full_service_history'].head()

0    Yes
1    Yes
2    Yes
3    Yes
4    Yes
Name: full_service_history, dtype: object

# Column Name : non_smoker_vehicle  #0-1 olarak düzenlenecek

In [291]:
df['non_smoker_vehicle'] = df['non_smoker_vehicle'].str.replace(',', '').str.strip()
df['non_smoker_vehicle'].value_counts()

non_smoker_vehicle
None    57769
Yes      4997
Name: count, dtype: int64

# Column Name : previous_owner  #0-1 olarak düzenlenecek

In [548]:
df['previous_owner'] = df['previous_owner'].str.lstrip("[' \s\S\t n ',]")
df['previous_owner'] = df['previous_owner'].str.rstrip("['\s,]")
df['previous_owner'].unique()

array(['None', '3', '2', '1', '5', '7', '9', '4', '6', '8', '12', '10'],
      dtype=object)

# Column Name : last_timing_belt_change  # 61925 None değeri hic degismedigi anlamina geldigi icin production date ile doldurulacak.

In [292]:
df['last_timing_belt_change'] = df['last_timing_belt_change'].str.lstrip(" [',] '\a \s\S\t\r n ' ")
df['last_timing_belt_change'] = df['last_timing_belt_change'].str.rstrip(" [',] '\a \s\S\t\r n ' ")

df['last_timing_belt_change'].unique()
df['last_timing_belt_change'].value_counts()

last_timing_belt_change
None       61925
06/2021       41
05/2021       31
10/2021       31
09/2021       28
           ...  
02/2015        1
04/2016        1
06/2014        1
04/2015        1
11/2015        1
Name: count, Length: 82, dtype: int64

# Column Name : Power   #dtype int e cevrilecek

In [550]:
df["power"].value_counts()

power
[None, '']         5333
110 kW (150 hp)    3677
88 kW (120 hp)     2518
100 kW (136 hp)    2190
90 kW (122 hp)     2174
                   ... 
378 kW (514 hp)       1
282 kW (383 hp)       1
470 kW (639 hp)       1
451 kW (613 hp)       1
10 kW (14 hp)         1
Name: count, Length: 361, dtype: int64

# Column Name: gearbox   aynı sütundan var

# Column Name : engine_size   # dtype int e cevrilecek

In [551]:
# kolonun ici düzenlendi, none ifade = 0 a ayarlandi.
# ancak cc biriminden dolayi int cevrilmiyor, bu yüzden de mean() vs islemlr de yapilamiyor
#??????

In [552]:
df["engine_size"]= df["engine_size"].str.strip(',\n,')
df["engine_size"]= df["engine_size"].replace("None", "0")
df["engine_size"]= df["engine_size"].str.replace("cc", "")

df["engine_size"].head()

0    1,984 
1    1,395 
2    1,395 
3    1,395 
4    1,495 
Name: engine_size, dtype: object

# Column Name : gears  # 18184 none degeri var literatür arastirmasi yapilacak  

In [553]:
df["gears"]=df["gears"].str.strip(',\n,')
df["gears"].unique()
df["gears"].

array(['None', '7', '6', '8', '5', '1', '9', '4', '2', '3'], dtype=object)

bi iki koda gerek var mi pek emin degilim,
Data missing de konusabiliriz!!!
df["gears"].replace("None", "0", inplace = True)
df["gears"]=df["gears"].astype(int)

# Column Name: cylinders  #literatür taraması yapılacak.

In [293]:
df["cylinders"]=df["cylinders"].str.strip(',\n,')
df["cylinders"].unique()
df["cylinders"].value_counts()

cylinders
4       41993
3        7664
None     5532
6        4260
5        1390
8         937
1         498
2         400
10         65
12         26
7           1
Name: count, dtype: int64

# Column Name : empty_weight

In [294]:
df["empty_weight"]=df["empty_weight"].str.strip('[\n,')
df["empty_weight"]=df["empty_weight"].str.replace(' kg', '')
df["empty_weight"].value_counts()

empty_weight
1,295    1529
None     1407
1,405    1127
1,635     795
1,065     658
         ... 
2,387       1
2,481       1
2,163       1
2,283       1
2,036       1
Name: count, Length: 1539, dtype: int64

# Column Name: Fuel_type   #bir sütun daha mevcut . Diğerindeki bilgiler daha kapsayıcı olduğu için bu sütun derop yapılacak.

In [556]:
df["Fuel_type"]=df["Fuel_type"].str.strip('[\n,')
df["Fuel_type"].head(15)

0     Super 95
1     Gasoline
2     Gasoline
3     Gasoline
4     Super 95
5     Super 95
6     Gasoline
7     Gasoline
8     Gasoline
9     Super 95
10    Gasoline
11    Super 95
12    Gasoline
13    Super 95
14    Gasoline
Name: Fuel_type, dtype: object

# Column name : fuel_consumption # city ve country sütunlari olusturulacak.

In [557]:
df['fuel_consumption'] = df['fuel_consumption'].str.lstrip(" [', \s\S\t\r\n,n ' ")
df['fuel_consumption'] = df['fuel_consumption'].str.rstrip("[', '] \s\S\t\r\n,\\n  ")
df['fuel_consumption'] = df['fuel_consumption'].str.replace("'","")
df['fuel_consumption']

0                                                     None
1                                     4.9 l/100 km (comb.)
2        0 l/100 km (comb.), 0 l/100 km (city), 0 l/100...
3        4.8 l/100 km (comb.), 5.9 l/100 km (city), 4.2...
4        5 l/100 km (comb.), 6.2 l/100 km (city), 4.3 l...
                               ...                        
62761                                                 None
62762                                                 None
62763    9.1 l/100 km (comb.), 12.6 l/100 km (city), 7 ...
62764                                                 None
62765    8.4 l/100 km (comb.), 11.6 l/100 km (city), 6....
Name: fuel_consumption, Length: 62766, dtype: object

# Column Name: co2_emissions

In [295]:
df['co2_emissions'] = df['co2_emissions'].str.strip(",\n")
df["co2_emissions"] = df["co2_emissions"].str.rstrip(" [',] \s\S\t\r\n n' ")
df["co2_emissions"] = df["co2_emissions"].str.lstrip(" [', \s\S\t\r\n ' ")
df['co2_emissions'].value_counts()


array(['', 'None', '111 g/km (comb.)', '113 g/km (comb.)',
       '0 g/km (comb.)', '122 g/km (comb.)', '149 g/km (comb.)',
       '127 g/km (comb.)', '136 g/km (comb.)', '102 g/km (comb.)',
       '195 g/km (comb.)', '110 g/km (comb.)', '115 g/km (comb.)',
       '112 g/km (comb.)', '159 g/km (comb.)', '139 g/km (comb.)',
       '88 g/km (comb.)', '116 g/km (comb.)', '169 g/km (comb.)',
       '103 g/km (comb.)', '109 g/km (comb.)', '129 g/km (comb.)',
       '180 g/km (comb.)', '114 g/km (comb.)', '106 g/km (comb.)',
       '130 g/km (comb.)', '216 g/km (comb.)', '187 g/km (comb.)',
       '175 g/km (comb.)', '119 g/km (comb.)', '174 g/km (comb.)',
       '192 g/km (comb.)', '137 g/km (comb.)', '124 g/km (comb.)',
       '172 g/km (comb.)', '230 g/km (comb.)', '190 g/km (comb.)',
       '152 g/km (comb.)', '179 g/km (comb.)', '166 g/km (comb.)',
       '194 g/km (comb.)', '189 g/km (comb.)', '148 g/km (comb.)',
       '170 g/km (comb.)', '147 g/km (comb.)', '108 g/km (comb.)',
      

# Column Name : emission_class # literatür taraması yapılacak ve Euro 6c ve 6d var mı kontrol edilecek. Yoksa bu değerlerin yeri Euro 6 ile doldurulacak.

In [296]:
df['emission_class'] = df['emission_class'].str.strip(",\n")
df['emission_class'].value_counts()

emission_class
Euro 6                           19704
None                             19076
Euro 5                           10675
Euro 4                            7710
Euro 6d-TEMP                      2675
Euro 3                            1702
['\n', [' ', ' ', ' '], '\n']      597
Euro 1                             461
Euro 2                             129
Euro 6c                             20
Euro 6d                             17
Name: count, dtype: int64

# Column Name: emissions_sticker   #Drop yapılabilir.

In [560]:
df["emissions_sticker"].unique()

array(['\n,None,\n', '\n,4 (Green),\n', '\n,3 (Yellow),\n',
       '\n,1 (No sticker),\n', "['\\n', [' ', ' ', ' '], '\\n']",
       '\n,2 (Red),\n'], dtype=object)

In [297]:
df["emissions_sticker"] = df["emissions_sticker"].str.rstrip(" [',] \s\S\t\r\n ' ")
df["emissions_sticker"] = df["emissions_sticker"].str.lstrip(" [',] \s\S\t\r\n ' ")
df["emissions_sticker"] = df["emissions_sticker"].str.strip(" ")
df["emissions_sticker"] = df["emissions_sticker"].str.replace("\\n","")
df["emissions_sticker"].value_counts()

emissions_sticker
None                      62632
4 (Green)                   106
1 (No sticker)               12
n', [' ', ' ', ' '], '       11
3 (Yellow)                    4
2 (Red)                       1
Name: count, dtype: int64

# Column Name : Comfort & Convenience  #Air conditioning ve Cruise control sütunları oluşturulacak ve 1-0 ile doldurulacak.

In [298]:
df.rename({"\nComfort & Convenience\n":"comfort_convenience"}, inplace = True, axis = 1) # sütun isminde \n değerleri olduğu için kalıcı oalrak sütun ismi değiştirildi.
df["comfort_convenience"] = df["comfort_convenience"].str.rstrip(" [', \s\S\t\r n ' ")
df["comfort_convenience"] = df["comfort_convenience"].str.lstrip(" [', '\a \s\S\t\r n ' ")
df["comfort_convenience"] = df["comfort_convenience"].str.replace("'","")
df["comfort_convenience"].value_counts()

comfort_convenience
Air conditioning, Power windows]]                                                                                                                                                                                                                                                                                                                                                                                                                               245
Air conditioning, Electrical side mirrors, Power windows]]                                                                                                                                                                                                                                                                                                                                                                                                      234
Air conditioning, Electrical side mirrors, Power windows, Tinted windows]]  

# Column Name : Entertaiment & Media Bluetooth Sütunu oluşturulacak. 1-0 ile doldurulacak

In [563]:
df['\nEntertainment & Media\n'].unique()

array(["[['CD player', 'Radio']]",
       "[['Bluetooth', 'CD player', 'On-board computer', 'USB']]",
       "[['CD player', 'MP3', 'On-board computer', 'Radio']]", ...,
       "[['Apple CarPlay', 'CD player', 'Digital cockpit', 'On-board computer', 'Radio']]",
       "[['Apple CarPlay', 'CD player', 'On-board computer', 'Radio', 'Sound system']]",
       "[['Android Auto', 'Apple CarPlay', 'Bluetooth', 'CD player', 'Digital cockpit', 'Digital radio', 'Hands-free equipment', 'Induction charging for smartphones', 'Integrated music streaming', 'On-board computer', 'Radio', 'Sound system', 'USB']]"],
      dtype=object)

In [564]:
df.rename({"\nEntertainment & Media\n":"entertainment_media"},inplace = True, axis = 1)# sütun isminde \n değerleri olduğu için kalıcı oalrak sütun ismi değiştirildi.
df["entertainment_media"] = df["entertainment_media"].str.rstrip(" [',] \s\S\t\r n ' ")
df["entertainment_media"] = df["entertainment_media"].str.lstrip(" [', '\a \s\S\t\r n ' ")
df["entertainment_media"] =  df["entertainment_media"].str.replace("'","")
df["entertainment_media"].unique()

array(['CD player, Radio', 'Bluetooth, CD player, On-board computer, USB',
       'CD player, MP3, On-board computer, Radio', ...,
       'Apple CarPlay, CD player, Digital cockpit, On-board computer, Radio',
       'Apple CarPlay, CD player, On-board computer, Radio, Sound system',
       'Android Auto, Apple CarPlay, Bluetooth, CD player, Digital cockpit, Digital radio, Hands-free equipment, Induction charging for smartphones, Integrated music streaming, On-board computer, Radio, Sound system, USB'],
      dtype=object)

# Column Name : Safety & Security  ABS için sütun oluşturulacak. 1-0 şeklinde doldurulacak.

In [565]:
df.rename({"\nSafety & Security\n":"safety_security"},inplace = True, axis = 1)# sütun isminde \n değerleri olduğu için kalıcı oalrak sütun ismi değiştirildi.
df["safety_security"] = df["safety_security"].str.rstrip(" [', \s\S\t\r n ' ")
df["safety_security"] = df["safety_security"].str.lstrip(" [', '\a \s\S\t\r n ' ")
df["safety_security"] =  df["safety_security"].str.replace("'","")
df["safety_security"]

0        ABS, Alarm system, Central door lock, Central ...
1        Alarm system, Central door lock, Central door ...
2        ABS, Central door lock with remote control, Dr...
3        ABS, Alarm system, Bi-Xenon headlights, Centra...
4        ABS, Alarm system, Bi-Xenon headlights, Centra...
                               ...                        
62761    ABS, Central door lock, Central door lock with...
62762    ABS, Central door lock, Central door lock with...
62763    ABS, Central door lock, Central door lock with...
62764    ABS, Central door lock with remote control, Dr...
62765    ABS, Alarm system, Central door lock, Central ...
Name: safety_security, Length: 62766, dtype: object

# Column Name : extras  #şimdilik bir değişiklik yapılmayacak

In [566]:
df["extras"] = df["extras"].str.rstrip(" [',] \s\S\t\r n ' ")
df["extras"] = df["extras"].str.lstrip(" [',] '\a \s\S\t\r n ' ")
df["extras"] =  df["extras"].str.replace("'","")
df["extras"]

0                                              Alloy wheel
1                                       Alloy wheels (16")
2                                       Alloy wheels (15")
3        Alloy wheels (17"), Shift paddles, Sport seats...
4         Alloy wheels (17"), Sport seats, Sport suspensio
                               ...                        
62761                                   Alloy wheels (15")
62762                                   Alloy wheels (15")
62763    Alloy wheels (17"), Automatically dimming inte...
62764    Alloy wheels (16"), Automatically dimming inte...
62765                                          Alloy wheel
Name: extras, Length: 62766, dtype: object

# Column Name: manifacturer_colour    içeriği tamamen nan değeriyle dolu

In [567]:
df['\nmanufacturer_color'].unique()

array([nan])

# Column Name: colour  # 1048 none değeri renklerin yüzdelik oranı değerinde doldurulacak.

In [568]:
df.rename(columns = {'\ncolour':'colour'}, inplace = True)
df["colour"].unique()

array(['\n4, Grey', '\n4, White', '\n4, Black', '\n4, Red', '\n4, Silver',
       '\n4, Brown', '\n4, Blue', '\n4, Violet', nan, '\n4, Beige',
       '\n4, Yellow', '\n4, Green', '\n4, Orange', '\n4, Bronze',
       '\n4, Gold'], dtype=object)

In [569]:
df["colour"]=df["colour"].str.strip(',\n4,')
df["colour"].head()
#1048 none var

0      Grey
1     White
2      Grey
3     Black
4      Grey
Name: colour, dtype: object

# Column Name : paint   (arabanın boyanıp boyanmadığını ifade ediyor. Renk belirtilenler boyanmış belirtilmeyenler boyanmamış. Bundan dolayı rengin yanındaki paint ifadesini gözardı edebiliriz.)

In [300]:
df.rename(columns = {'\npaint':'paint'}, inplace = True)
df["paint"].unique()
df['paint']


0        \npaint, Metallic
1                      NaN
2        \npaint, Metallic
3                      NaN
4        \npaint, Metallic
               ...        
62761                  NaN
62762                  NaN
62763    \npaint, Metallic
62764    \npaint, Metallic
62765    \npaint, Metallic
Name: paint, Length: 62766, dtype: object

In [301]:
df["paint"]=df["paint"].str.replace('\npaint,','')
df['paint'].value_counts()

paint
 Metallic    44423
Name: count, dtype: int64

# Column Name : upholstery_colour    #literatür taraması yapılacak

In [30]:
df[' upholstery_colour '].value_counts()

 upholstery_colour 
\nupholstery, Black     30365
\nupholstery, Grey       5386
\nupholstery, Beige      1600
\nupholstery, Other      1484
\nupholstery, Brown      1145
\nupholstery, Red         296
\nupholstery, Blue        293
\nupholstery, White       264
\nupholstery, Orange       22
\nupholstery, Yellow       15
\nupholstery, Green        10
Name: count, dtype: int64

In [573]:
df.rename(columns={' upholstery_colour ':'upholstery_colour'},inplace = True)
df["upholstery_colour"]=df["upholstery_colour"].str.replace('\nupholstery,','')
df["upholstery_colour"].unique()

array([' Black', nan, ' Grey', ' Brown', ' Other', ' Beige', ' Red',
       ' Yellow', ' Blue', ' White', ' Green', ' Orange'], dtype=object)

# Column Name : upholstery  # literatür tararması yapılacak

In [574]:
df[" upholstery "].unique()

array(['\n8, Cloth', nan, '\n8, Part leather', '\n8, Full leather',
       '\n8, alcantara', '\n8, Other', '\n8, Velour'], dtype=object)

In [31]:
df.rename(columns={' upholstery ':'upholstery'},inplace = True)
df["upholstery"]=df["upholstery"].str.replace('\n8,','')
df["upholstery"].value_counts()

upholstery
 Cloth           23375
 Full leather    17346
 Part leather     9492
 alcantara         972
 Other             481
 Velour            417
Name: count, dtype: int64

In [284]:
#df['registration'] = df['fuel_type'].str.strip("\n\r,")
#df['Fuel_type'] = df['Fuel_type'].str.strip("\n\r,")
#df['registration'] = df['registration'].fillna("None")
df.loc[(df['production_date'] != 'None'), ['registration', 'production_date']].head(50)
#df['fuel_type'].value_counts() # Icinde gearbox ile lgili bilgiler de var. Kayit ederken kaydirma yapilmis olabilir
# fuel_type i secmek digerini silmek mantkli olabilir


,registration,production_date
0,08/2003,NaN
1,10/2013,NaN
2,06/2015,2015.0
3,10/2015,NaN
4,10/2017,NaN
5,03/2019,NaN
6,03/2014,NaN
7,07/2011,2011.0
8,08/2012,2012.0
9,01/2011,NaN
